# Speeding up your code

**Table of contents**<a id='toc0_'></a>    
- 1. [Computers](#toc1_)    
  - 1.1. [Three important principles](#toc1_1_)    
- 2. [Timing and precomputations](#toc2_)    
- 3. [Line profilling](#toc3_)    
- 4. [List comprehensions are your friend](#toc4_)    
- 5. [Generators](#toc5_)    
- 6. [Optimizing Numpy](#toc6_)    
  - 6.1. [Tip 1: Always use vectorized operations when available](#toc6_1_)    
  - 6.2. [Tip 2: Operations are faster on rows than on columns](#toc6_2_)    
  - 6.3. [Tip 3: Also use vectorized operations when it is a bit cumbersome](#toc6_3_)    
- 7. [Summary](#toc7_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

You will learn how to time your code and locate its bottlenecks. 

You will learn how to alleviate such bottlenecks using techniques such as **comprehensions**, **generators** and **vectorization**. 

You will hear about the fundamental computational costs of mathematical operations and memory management (caching).

In [1]:
%load_ext autoreload
%autoreload 2

import time
import numpy as np
from scipy import optimize

import matplotlib.pyplot as plt
plt.rcParams.update({"axes.grid":True,"grid.color":"black","grid.alpha":"0.25","grid.linestyle":"--"})
plt.rcParams.update({'font.size': 14})

# magics
#  conda install line_profiler
#  conda install memory_profiler

%load_ext line_profiler
%load_ext memory_profiler

# local module
import needforspeed

## 1. <a id='toc1_'></a>[Computers](#toc0_)

We can represent a **computer** in a simplified diagram as:

<img src="computer.gif" alt="computer" width=60% />

**Performance goals:**

1. Minimize the number of logical and algebraic operations ([details](https://streamhpc.com/blog/2012-07-16/how-expensive-is-an-operation-on-a-cpu/))
2. Minimize the number of times new memory needs to be allocated (and the amount)
3. Minimize the number of read and write memory (and especially storage) operations

Optimizing your code for **optimal performance is a very very complicated task**. 

When using Python a lot of stuff is happening *under the hood*, which you don't control. 

* Python is an **interpreted** language; each line of Python code is converted into machine code at runtime when the line is reached. Error checks and memory management are performed automatically.
* Faster languages (C/C++, Fortran) are **compiled** to machine code before the program is run $\rightarrow$ faster, but you are required to specify e.g. types of variables beforehand. Error checks and memory management must be performed manually.

**Often overlooked**, todays CPUs are so fast that feeding them data quickly enough can be a serious bottleneck.

**Modern CPUs** can do a lot of smart, complicated, stuff.

* **Single-instruction multiply data (SIMD):** The computional cost of multiplying one float with another is the same as multiplying e.g. vectors of 4 doubles at once (or 8 doubles if you have AVX-512).

* **Out-of-order execution:** If you tell the computer to
 
    1. read data ``X``
    2. run ``f(X)``
    3. read data ``Y``
    4. run ``g(Y)``

    then it might try to do step 2 and step 3 simultanously because they use different parts of the CPU.

* **Caching:** Let ``x`` be a one-dimensional numpy array, and assume you read the value in ``x[i]`` and then read the value in ``x[j]``. If ``j`` is "close" to ``i`` then the value of ``x[j]`` will already be in the *cache* and the second read operation will be faster (almost instantanous).

**Parallization:** 

1. Modern computers have multiple CPUs (or even other computing units such as GPUs). 
2. This is to some degree used implicitely by e.g. built-in Numpy and Scipy functions, but can also be done manually. 
3. The clock speed of each CPU has stopped increasing for technical reasons, the number of transistors on each chip continue to increase exponentially (**Moore's Law**) due to more CPUs.

<img src="moores_law.png" alt="moores_law" width=80% />

**Memory:** We have many different kinds of memory

1. Cache
2. RAM (Random Access Memory)
3. Hard drive

We control what is in the **RAM** and on the the **hard drive**; the latter is a lot slower than the former.
<br>The cache is used by the computer under the hood.

<img src="memory.gif" alt="memory" width=40% />

### 1.1. <a id='toc1_1_'></a>[Three important principles](#toc0_)

1. **Use built-in features** of Python, Numpy, Scipy etc. whenever possible (often use fast compiled code).
2. **Ordered operations** is better than random operations.
3. **"Premature optimization is the root of all evil"** (Donald Knuth). 

There is a **trade-off** between **human time** (the time it takes to write the code) and **computer time** (the time it takes to run the code).

## 2. <a id='toc2_'></a>[Timing and precomputations](#toc0_)

Consider the following function doing some simple algebraic operations:

In [2]:
def myfun(x,i):
    y = 0
    for j in range(100):
        y += x**j
    return y + i

And another function calling the former function in a loop:

In [3]:
def myfun_loop(n):
    mysum = 0
    for i in range(n):
        mysum += myfun(5,i)
    return mysum

**How long does it take to run ``myfun_loop``:**

**A.** Manual timing

In [4]:
t0 = time.time()
mysum = myfun_loop(1000)
t1 = time.time()    
print(f'{t1-t0:.8} seconds')

0.04897356 seconds


**B.** Use the ``%time`` magic (work on a single line)

In [5]:
%time mysum = myfun_loop(1000)
%time mysum = myfun_loop(1000)

Wall time: 83 ms
Wall time: 77 ms


**ms** $\equiv$ milliseconds, $10^{-3}$ of a second.<br>
**$\mu$** $\equiv$ mikroseconds, $10^{-6}$ of a second.<br>
**ns** $\equiv$ nanoseconds, $10^{-9}$ of a second.

**C.** Use the ``%timeit`` magic to also see variability (work on single line)

In [6]:
%timeit myfun_loop(1000)
%timeit -r 5 -n 20 myfun_loop(1000)

60.4 ms ± 21.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
42.3 ms ± 732 µs per loop (mean ± std. dev. of 5 runs, 20 loops each)


``%timeit`` report the best of ``r`` runs each calling the code ``n`` times in a loop

**D.** Use the ``%%time`` magic (work on a whole cell)

In [7]:
%%time
n = 1000
myfun_loop(n);

Wall time: 44 ms


1972152263052529513529321413206965574183016087772557511925697326660655500

**E.** Use the ``%%timeit`` magic to also see variabilty (work on a whole cell)

In [8]:
%%timeit
n = 1000
myfun_loop(n)

44.4 ms ± 553 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


**Question:** How can we speed up the computation using **precomputation**?

In [9]:
def myfun_loop_fast(n):
    myfunx = myfun(5,0)
    mysum = 0
    for i in range(n):
        mysum = myfunx+i
    return mysum
    
# remember
def myfun_loop(n):
    mysum = 0
    for i in range(n):
        mysum += myfun(5,i)
    return mysum

def myfun(x,i):
    y = 0
    for j in range(100):
        y += x**j
    return y + i

**Answer:**

In [10]:
def myfun_loop_fast(n):
    myfunx = myfun(5,0) # precomputation
    mysum = 0
    for i in range(n):
        mysum += myfunx + i
    return mysum

In [11]:
t0 = time.time()
mysum_fast = myfun_loop_fast(1000)
t1 = time.time()    
print(f'{t1-t0:.8f} seconds')

0.00000000 seconds


Too fast to be measured with ``time.time()``. The ``%timeit`` magic still works:

In [12]:
%timeit myfun_loop(1000)
%timeit myfun_loop_fast(1000)

43.7 ms ± 722 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
144 µs ± 6.72 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


$\rightarrow$ **orders of magnitude faster!**

Check the **results are the same**:

In [13]:
assert mysum == mysum_fast

## 3. <a id='toc3_'></a>[Line profilling](#toc0_)

**Premature optimization is the root of all evil!**

**Important:** Before deciding whether to do a precomputation (which often makes the code harder to read) we should investigate, whether it alleviates a bottleneck.

* **A.** Insert multiple ``time.time()`` to time different parts of the code.
* **B.** Use the ``line_profiler`` with syntax (also works with methods for classes)

  ``%lprun -f FUNCTION_TO_PROFILE -f FUNCTION_TO_PROFILE FUNCTION_TO_RUN``

**Baseline method:**

In [14]:
%lprun -f myfun -f myfun_loop myfun_loop(1000)

Timer unit: 1e-07 s

Total time: 0.162366 s

Could not find file C:\Users\gmf123\AppData\Local\Temp\ipykernel_8784\2119978152.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
     9                                           
    10         1         15.0     15.0      0.0  
    11      1001       3338.0      3.3      0.2  
    12      1000    1620305.0   1620.3     99.8  
    13         1          3.0      3.0      0.0  

Total time: 0.110838 s

Could not find file C:\Users\gmf123\AppData\Local\Temp\ipykernel_8784\2119978152.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
    15                                           
    16      1000       3335.0      3.3      0.3  


**Observation:** Most of the time is spend in ``myfun()``, more specifically the computation of the power in line 4. The precomputation solves this problem.

**Compare with the fast method:**

In [15]:
%lprun -f myfun_loop_fast myfun_loop_fast(1000)

Timer unit: 1e-07 s

Total time: 0.0008266 s

Could not find file C:\Users\gmf123\AppData\Local\Temp\ipykernel_8784\3913433440.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           
     2         1        878.0    878.0     10.6  
     3         1          7.0      7.0      0.1  
     4      1001       3045.0      3.0     36.8  
     5      1000       4333.0      4.3     52.4  
     6         1          3.0      3.0      0.0

## 4. <a id='toc4_'></a>[List comprehensions are your friend](#toc0_)

We can find the first $n$ squares using a **loop**:

In [16]:
def squares(n):
    result = []
    for i in range(n):
        result.append(i*i)
    return result

Or in a **list comprehension**:

In [17]:
def squares_comprehension(n):
    return [i*i for i in range(n)]

They give the **same result**:

In [18]:
n = 1000
mylist = squares(n)
mylist_fast = squares_comprehension(n)
assert mylist == mylist_fast

But the **list comphrension is faster**:

In [19]:
%timeit mylist = squares(n)
%timeit mylist_fast = squares_comprehension(n)

99.7 µs ± 5.69 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
66.9 µs ± 5.1 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


**Question:** Why is this slower?

In [20]:
%timeit [i**2 for i in range(1,n+1)]

273 µs ± 10.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## 5. <a id='toc5_'></a>[Generators](#toc0_)

Assume you are only interested in the **sum of the squares**. Can be calculated as follows:

In [21]:
squares_list = [i*i for i in range(n)]
mysum = 0
for square in squares_list:
    mysum += square

**Problem:** In line 1 we create the full list even though we only need one element at a time<br>
$\rightarrow $ *we allocate memory we need not allocate.*

**Solution:** Can be avoided with a **generator**.

In [22]:
squares_generator = (i*i for i in range(n)) # notice: parentheses instead of brackets
mysum_gen = 0
for square in squares_generator:
    mysum_gen += square

assert mysum == mysum_gen

The **memory footprint** can be investigated with the **memory_profiler** with syntax

``%mprun -f FUNCTION_TO_PROFILE -f FUNCTION_TO_PROFILE FUNCTION_TO_RUN``

**Caveat:** Needs to be a function in an external module.

In [23]:
%mprun -f needforspeed.test_memory needforspeed.test_memory(10**6)

Filename: c:\Users\gmf123\Dropbox\Repositories\course\IntroProg-lectures\4 - Speed\needforspeed.py

Line #    Mem usage    Increment  Occurences   Line Contents
     9    121.5 MiB    121.5 MiB           1   def test_memory(n):
    10                                         
    11                                             # list vs. generators
    12    160.2 MiB     38.7 MiB           1       squares = create_list(n)
    13    160.2 MiB      0.0 MiB           1       squares_generator = create_generator(n)
    14                                         
    15                                             # numpy arrays of different types
    16    167.8 MiB      7.6 MiB           1       A = np.ones((1000,1000))
    17    175.5 MiB      7.6 MiB           1       B = np.ones((1000,1000),dtype=np.double)
    18    179.3 MiB      3.8 MiB           1       C = np.ones((1000,1000),dtype=np.single)
    19    186.9 MiB      7.6 MiB           1       D = np.ones((1000,1000),dtype=np.int64)


 **MiB** 1 MiB = 1.048576 MB

 **Numpy:** Note how you can save memory by specifying the data type for the numpy array.

**Alternative:** Generators can also be created as functions with a ``yield`` instead of a ``return`` 

In [24]:
def f_func(n):
    for i in range(n):
        yield i*i

squares_generator = f_func(n)
mysum_gen = 0
for square in squares_generator:
    mysum_gen += square

assert mysum == mysum_gen

## 6. <a id='toc6_'></a>[Optimizing Numpy](#toc0_)

### 6.1. <a id='toc6_1_'></a>[Tip 1: Always use vectorized operations when available](#toc0_)

**Simple comparison:**

In [25]:
x = np.random.uniform(size=500000)

def python_add(x):
    y = []
    for xi in x:
        y.append(xi+1)
    return y

def numpy_add(x):
    y = np.empty(x.size)
    for i in range(x.size):
        y[i] = x[i]+1
    return y

def numpy_add_vec(x):
    return x+1

assert np.allclose(python_add(x),numpy_add(x))
assert np.allclose(python_add(x),numpy_add_vec(x))

%timeit python_add(x)
%timeit numpy_add(x)
%timeit numpy_add_vec(x)

175 ms ± 3.72 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
207 ms ± 6.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
1.7 ms ± 107 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Even **stronger** when the **computation is more complicated:**

In [26]:
def python_exp(x):
    y = []
    for xi in x:
        y.append(np.exp(xi))
    return y

def numpy_exp(x):
    y = np.empty(x.size)
    for i in range(x.size):
        y[i] = np.exp(x[i])
    return y

def numpy_exp_vec(x):
    return np.exp(x)

assert np.allclose(python_exp(x),numpy_exp(x))
assert np.allclose(python_exp(x),numpy_exp_vec(x))

%timeit python_exp(x)
%timeit numpy_exp(x)
%timeit numpy_exp_vec(x)

573 ms ± 7.48 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
677 ms ± 43.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
4.85 ms ± 715 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Also works for a **conditional sum**:

In [27]:
def python_exp_cond(x):
    return [np.exp(xi) for xi in x if xi < 0.5]

def numpy_exp_vec_cond(x):
    y = np.exp(x[x < 0.5])
    return y

def numpy_exp_vec_cond_alt(x):
    y = np.exp(x)[x < 0.5]
    return y

assert np.allclose(python_exp_cond(x),numpy_exp_vec_cond(x))
assert np.allclose(python_exp_cond(x),numpy_exp_vec_cond_alt(x))

%timeit python_exp_cond(x)
%timeit numpy_exp_vec_cond(x)
%timeit numpy_exp_vec_cond_alt(x)

359 ms ± 30.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
7.37 ms ± 704 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
8.14 ms ± 998 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


**Question:** Why do you think the speed-up is less pronounced in this case?

### 6.2. <a id='toc6_2_'></a>[Tip 2: Operations are faster on rows than on columns](#toc0_)

Generally, operate on the **outermost index**.

In [28]:
n = 1000
x = np.random.uniform(size=(n,n))

def add_rowsums(x):
    mysum = 0
    for i in range(x.shape[0]):
        mysum += np.sum(np.exp(x[i,:]))
    return mysum
            
def add_colsums(x):
    mysum = 0
    for j in range(x.shape[1]):
        mysum += np.sum(np.exp(x[:,j]))
    return mysum

assert np.allclose(add_rowsums(x),add_colsums(x))
            
%timeit add_rowsums(x)
%timeit add_colsums(x)

13.5 ms ± 289 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
25 ms ± 328 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


<img src="https://github.com/NumEconCopenhagen/lectures-2019/raw/master/11/numpy_memory_layout.png" alt="amdahls_law" width=60% />

The **memory structure can be changed manually** so that working on columns (innermost index) is better than working on rows (outermost index):

In [29]:
y = np.array(x,order='F') # the default is order='C'
%timeit add_rowsums(y)
%timeit add_colsums(y)

25 ms ± 535 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
14.6 ms ± 977 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### 6.3. <a id='toc6_3_'></a>[Tip 3: Also use vectorized operations when it is a bit cumbersome](#toc0_)

Consider the task of calculating the following **expected value**:

$$
\begin{aligned}
W(a)&=\mathbb{E}\left[\sqrt{\frac{a}{\psi}+\xi}\right]\\
\psi,\xi&\in \begin{cases}
0.25 & \text{with prob. }0.25\\
0.5 & \text{with prob. }0.25\\
1.5 & \text{with prob. }0.25\\
1.75 & \text{with prob. }0.25
\end{cases}\end{aligned}
$$

for a vector of $a$-values.

**Setup:**

In [30]:
N = 5000
a_vec = np.linspace(0,10,N)

xi_vec = np.array([0.25,0.5,1.5,1.75])
psi_vec = np.array([0.25,0.5,1.5,1.75])

xi_w_vec = np.ones(4)/4
psi_w_vec = np.ones(4)/4

**Loop based solution:**

In [31]:
def loop(a_vec,xi_vec,psi_vec,xi_w_vec,psi_w_vec):
    
    w_vec = np.zeros(a_vec.size)
    for i,a in enumerate(a_vec):        
        for xi,xi_w in zip(xi_vec,xi_w_vec):
            for psi,psi_w in zip(psi_vec,psi_w_vec):
                m_plus = a/psi + xi
                v_plus = np.sqrt(m_plus)
                w_vec[i] += xi_w*psi_w*v_plus
    
    return w_vec
        
loop_result = loop(a_vec,xi_vec,psi_vec,xi_w_vec,psi_w_vec)  
%timeit loop(a_vec,xi_vec,psi_vec,xi_w_vec,psi_w_vec)      

188 ms ± 8.26 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


**Vectorized solution:**

In [32]:
def vec(a,xi,psi,xi_w,psi_w):   
    m_plus_vec = a[:,np.newaxis,np.newaxis]/psi[np.newaxis,:,np.newaxis] + xi[np.newaxis,np.newaxis,:]
    v_plus_vec = np.sqrt(m_plus_vec)
    w_mat = xi_w[np.newaxis,np.newaxis,:]*psi_w[np.newaxis,:,np.newaxis]*v_plus_vec
    w_vec = np.sum(w_mat,axis=(1,2))
    return w_vec

vec_result = vec(a_vec,psi_vec,xi_vec,xi_w_vec,psi_w_vec)
assert np.allclose(loop_result,vec_result)
%timeit vec(a_vec,psi_vec,xi_vec,xi_w_vec,psi_w_vec)

1.42 ms ± 31.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


**Conclusion:** Much much faster.

## 7. <a id='toc7_'></a>[Summary](#toc0_)

You learned that optimizing performance is a difficult task, but the recommendation is to follow the following 4-step procedure:

1. Choose the **right algorithm**
2. Implement **simple and robust code** for the algorithm 
3. Profile the code to **find bottlenecks**
4. Use **precomputations**, **comphrensions** and **vectorization** to speed-up the code